# Get the Weather Description and Amount of Precipitation for Each City

In [1]:
# Import the dependencies.
import pandas as pd
import numpy as np

In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [4]:
from citipy import citipy

In [5]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count.
len(cities)

620

In [6]:
# Import the datetime module from the datetime library.
from datetime import datetime

In [7]:
# Import the requests library.
import requests

# Import the API key.
from config import weather_api_key

In [8]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=4207db3a377792eb24c750378dd88d03


In [9]:
# Create an empty list to hold the weather data.
city_data = []

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    city_url = url + "&q=" + city

  
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Parse out the needed data, adding dictionary index since it is nested.
        city_weather_description = city_weather["weather"][0]["description"]
        
        # Get the amount of rainfall in inches for the last 3 hours
        try:
            city_rain_inches = city_weather["rain"]["3h"] / 25.4
        
        except KeyError:
            city_rain_inches = 0
            
        # Get the amount of snowfall in inches for the last 3 hours
        try:
            city_snow_inches = city_weather["snow"]["3h"] / 25.4
        
        except KeyError:
            city_snow_inches = 0
                
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        
        # Convert rainfall and snowfall to inches
        
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Date": city_date,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": city_weather_description,
                          "Rain inches (last 3 hrs)": city_rain_inches,
                          "Snow inches (last 3 hrs)": city_snow_inches})

    # If an error is experienced, skip the city.
    except:
        pass

In [10]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
0,Ambanja,MG,2020-04-03 17:08:01,-13.68,48.45,79.16,76,40,0.87,light rain,0.066142,0.0
1,Qingquan,CN,2020-04-03 17:08:02,30.45,115.26,56.77,77,94,2.53,overcast clouds,0.000000,0.0
2,Mataura,NZ,2020-04-03 17:08:02,-46.19,168.86,46.99,96,99,2.30,overcast clouds,0.000000,0.0
3,Mindelo,CV,2020-04-03 17:08:02,16.89,-24.98,82.40,51,20,21.92,few clouds,0.000000,0.0
4,Conceicao Do Araguaia,BR,2020-04-03 17:08:02,-8.26,-49.26,91.40,59,75,3.36,broken clouds,0.000000,0.0
5,Ushuaia,AR,2020-04-03 17:04:59,-54.80,-68.30,47.97,51,43,8.25,light rain,0.004331,0.0
6,Ulaanbaatar,MN,2020-04-03 17:08:03,47.91,106.88,23.00,57,0,2.24,clear sky,0.000000,0.0
7,Enkoping,SE,2020-04-03 17:08:03,59.64,17.08,42.80,26,63,17.22,broken clouds,0.000000,0.0
8,Hermanus,ZA,2020-04-03 17:08:03,-34.42,19.23,73.00,83,64,6.76,broken clouds,0.000000,0.0
9,Constitucion,CL,2020-04-03 17:08:03,-35.33,-72.42,70.36,38,24,12.68,few clouds,0.000000,0.0


In [11]:
# Create the output file (CSV).
output_challenge_data_file = "data/WeatherPy_challenge.csv"

# Export the City_Data into a CSV.
city_data_df.to_csv(output_challenge_data_file, index=False)

In [12]:
# Create DataFrame to determine how many cities have recorded rainfall or snowfall in the last 3 hours.
cities_rain_snow_df = city_data_df.loc[(city_data_df["Rain inches (last 3 hrs)"] > 0) |
                                       (city_data_df["Snow inches (last 3 hrs)"] > 0)]

# Get the number of cies that have recorded rainfall or snowfall in the last 3 hours.
len(cities_rain_snow_df)

98